In [ ]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
from torch.utils.data import Dataset

from tqdm.notebook import trange, tqdm

In [ ]:
# DATASET
class LSTMDataset(Dataset):
    def __init__(self, data, labels, sequence_length, transform=None):
        self.data = data
        self.labels = labels
        self.sequence_length = sequence_length

        self.transform = transform #TODO do transforms here (somehow)

        # TODO: Convert to torch data instead

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        # X: shape (sequence_length, num_features)
        X = self.data[idx : idx + self.sequence_length]
        # y: label at the next time step
        y = self.labels[idx + self.sequence_length]
        return X, y

In [ ]:
df = pd.read_csv('')

In [ ]:
train_loaders = None
test_loaders = None

## Training

In [ ]:
from LSTM import LSTM
from LSTM  import LSTMTrainer

In [ ]:
# MODEL
model = LSTM(input_size=10, hidden_size=64, num_layers=1, dropout=0.2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

trainer = LSTMTrainer(model, train_loaders, test_loaders, optimizer, criterion, num_epochs=10)

In [ ]:
# TRAINING
trainer.train()